# Lab03_108032053

## 1. Load dataset

In [51]:
%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import display

import os
if not os.path.exists("output/"):
  os.mkdir("output/")

import warnings
warnings.filterwarnings("ignore")

In [52]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = pd.DataFrame(data=data['data'], columns=data['feature_names'])
y = pd.DataFrame(data=data['target'], columns=['label'])

display(X.head(3))


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [53]:
from sklearn.model_selection import train_test_split

# Split dataset to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## 2. Use PCA to decrease # features

In [54]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Get the important feature
N = 2
pca = PCA(n_components=N)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Training and predict
forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=200, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train_pca, y_train)

y_pred_pca = forest.predict(X_test_pca)
print(f'Accuracy: {accuracy_score(y_test, y_pred_pca):.5f}')
print(f'Accuracy per feature: {accuracy_score(y_test, y_pred_pca) / X_train_pca.shape[1]:.5f}')

Accuracy: 0.94737
Accuracy per feature: 0.47368


## 3. Use Forest to decrease # features

In [55]:
from sklearn.feature_selection import SelectFromModel

forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=200, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train, y_train)

# Get the important feature
N = 2
sfm = SelectFromModel(forest, threshold=-np.inf, max_features=N)
sfm.fit(X_train, y_train)
X_train_forest = sfm.transform(X_train)
X_test_forest = sfm.transform(X_test)

# Training and predict
forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=200, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train_forest, y_train)

y_pred_forest = forest.predict(X_test_forest)
print(f'Accuracy: {accuracy_score(y_test, y_pred_forest):.5f}')
print(f'Accuracy per feature: {accuracy_score(y_test, y_pred_forest) / X_train_forest.shape[1]:.5f}')

Accuracy: 0.88889
Accuracy per feature: 0.44444
